La compañía de petroleo Oily Giant busca encontrar los 200 mejores lugares para abir pozos petroleros. 

In [1]:
#Cargar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats as st
from numpy.random import RandomState
#modelos, división de datos y métricas de evaluación
import sklearn
from sklearn.linear_model import LinearRegression
from  sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score, roc_auc_score, roc_curve, auc, recall_score, precision_score, mean_squared_error, r2_score
from sklearn.utils import resample

In [2]:
#Importación de datos
datos_geo1 = pd.read_csv("geo_data_0.csv")
datos_geo2 = pd.read_csv("geo_data_1.csv")
datos_geo3 = pd.read_csv("geo_data_2.csv")


In [3]:
#Exploración 
datos_geo1.info()
datos_geo2.info()
datos_geo3.info()

datos_geo1.describe()
datos_geo2.describe()
datos_geo3.describe()

display(datos_geo1.head())
display(datos_geo2.head())
display(datos_geo3.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null 

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Estado de los datos

Datos completos (sin nulos) en tipo de dato correcto. 

In [4]:
#División de datos
def division_datos(df, frac=0.25, rs=12345):
    """_summary_
    Divide los datos en entrenamiento y validación. 
    Genera 2 marcos de datos para entrenamiento y validación (4 totales): caracteristicas y objetivo. 
    Args:
        df (_type_): marco de datos a dividir.
        frac (float, optional): Fracción de datos que corresponde a la porción de validación. Defaults to 0.25.
        rs (int, optional): Estado al azar, garantiza reproducibilidad. Defaults to 12345.
    """
    train, valid = train_test_split(df, test_size=frac, random_state=rs)
    caracteristicas_entrenamiento = train.drop(columns = ["product", "id"]) #se retira id porque no tiene potencial predictivo
    objetivo_entrenamiento = train["product"] 
    caracteristicas_validacion = valid.drop(columns = ["product", "id"])
    objetivo_validacion = valid["product"]
    return caracteristicas_entrenamiento, objetivo_entrenamiento, caracteristicas_validacion, objetivo_validacion
    
    

In [5]:
#Entrenar el modelo
def entrenar_y_predecir(caracteristicas_entrenamiento, objetivo_entrenamiento, caracteristicas_validacion, objetivo_validacion, rs = 12345):
    """_summary_
    Entrena un modelo de regresión lineal. 
    Predice los valores de datos de validación. 
    Args:
        caracteristicas_entrenamiento (_type_): _description_
        objetivo_entrenamiento (_type_): _description_
        caracteristicas_validacion (_type_): _description_
        objetivo_validacion (_type_): _description_
        rs (int, optional): _description_. Defaults to 12345.
    """
    model = LinearRegression()
    model.fit(caracteristicas_entrenamiento, objetivo_entrenamiento)
    predicciones = model.predict(caracteristicas_validacion)
    prediccion_promedio = predicciones.mean()
    mse = mean_squared_error(objetivo_validacion, predicciones)
    rmse = np.sqrt(mse)
    return predicciones, prediccion_promedio, rmse
    

In [6]:
#Muestra el volumen medio de reservas predicho y RMSE del modelo. Para cada dataset: datos_geo1 y datos_geo2. 

#Ejecutar las funciones

#datos_geo1
#Division de datos
datos_divididos_geo1 = division_datos(datos_geo1, frac=0.25)
#Modelo. 
prediccion_y_rmse_geo1 = entrenar_y_predecir(*datos_divididos_geo1)

#datos_geo2
#Division de datos. 
datos_divididos_geo2 = division_datos(datos_geo2, frac=0.25)
#Modelo 
prediccion_y_rmse_geo2 = entrenar_y_predecir(*datos_divididos_geo2)

#datos_geo3
#Division de datos. 
datos_divididos_geo3 = division_datos(datos_geo3, frac=0.25)
#Modelo 
prediccion_y_rmse_geo3 = entrenar_y_predecir(*datos_divididos_geo3)

In [7]:
display(datos_divididos_geo1)

(             f0        f1         f2
 27212  0.022450  0.951034   2.197333
 7866   1.766731  0.007835   6.436602
 62041  0.724514  0.666063   1.840177
 70185 -1.104181  0.255268   2.026156
 82230 -0.635263  0.747990   6.643327
 ...         ...       ...        ...
 4094   1.863680 -0.298123   1.621324
 85412 -1.162682 -0.014822   6.819941
 2177   0.862688 -0.403776   1.867662
 77285  0.846235 -0.489533   1.058786
 86498  2.019850  0.263887  11.497239
 
 [75000 rows x 3 columns],
 27212    147.370612
 7866     147.630053
 62041     77.696728
 70185     55.210501
 82230    113.891723
             ...    
 4094     124.380793
 85412    144.874913
 2177     134.967255
 77285     64.494357
 86498    151.514894
 Name: product, Length: 75000, dtype: float64,
              f0        f1        f2
 71751  0.948970 -0.057547  2.095727
 80493  0.992974  0.206671 -0.142278
 2655   1.199854 -0.563356 -1.852991
 53233  0.691422 -0.433347  0.564974
 91141  0.420772  0.972638  0.736190
 ...         ..

In [8]:
#Exploración de datos de los modelos 
display(prediccion_y_rmse_geo1[1:3])
print("El promedio de producto (miles de barriles) en la zona geo1 es de 92.59 y varía en 37.58 unidades de producto de acuerdo al error cuadrado medio.")
display(prediccion_y_rmse_geo2[1:3])
print("El promedio de producto (miles de barriles) en la zona geo2 es de 68.73 y varía en 0.89 unidades.")
display(prediccion_y_rmse_geo3[1:3])
print("El promedio de producto (miles de barriles) en la zona geo3 es de 94.97 y varía en 40.02 unidades.")

print("Elegir entre las regiones con la cantidad de unidades de variación y promedio puede ser complicado así que se pondrá a prueba con muestreos múltiples: bootstrapping.")

(np.float64(92.59256778438035), np.float64(37.5794217150813))

El promedio de producto (miles de barriles) en la zona geo1 es de 92.59 y varía en 37.58 unidades de producto de acuerdo al error cuadrado medio.


(np.float64(68.72854689544602), np.float64(0.8930992867756165))

El promedio de producto (miles de barriles) en la zona geo2 es de 68.73 y varía en 0.89 unidades.


(np.float64(94.96504596800489), np.float64(40.02970873393434))

El promedio de producto (miles de barriles) en la zona geo3 es de 94.97 y varía en 40.02 unidades.
Elegir entre las regiones con la cantidad de unidades de variación y promedio puede ser complicado así que se pondrá a prueba con muestreos múltiples: bootstrapping.


In [9]:
#Bootstrapping. ¿Cuál es la mejor región petrolífera? 
#Definir función bootstrap para obtener promedio más certero. 
def bootstrap_promedio_region(y, n_iteraciones=1000, seed=42):
    np.random.seed(seed)
    medias = []

    y_array = y.to_numpy()

    for _ in range(n_iteraciones):
        muestra = resample(y_array, replace=True, n_samples=len(y_array))
        medias.append(np.mean(muestra))

    promedio_est = np.mean(medias)
    intervalo = np.percentile(medias, [2.5, 97.5])
    
    return promedio_est, intervalo

In [18]:
#Aplicar la función bootstrap a las regiones
prom_region1 = bootstrap_promedio_region(datos_divididos_geo1[1])
prom_region2 = bootstrap_promedio_region(datos_divididos_geo2[1])
prom_region3 = bootstrap_promedio_region(datos_divididos_geo3[1])

print(f"Promedio de producto en región 1: {prom_region1[0]}, \nintervalos de confianza región 1: 95%: {prom_region1[1]}\n")
print(f"Promedio de producto en región 2: {prom_region2[0]}, \nintervalos de confianza región 2 95%: {prom_region2[1]}\n")
print(f"Promedio de producto en región 3: {prom_region3[0]}, \nintervalos de confianza región 3 95%: {prom_region3[1]}\n")
#Seleccionar región con beneficio promedio mayor
print("La región 3 es la más petrolífera con 95 unidades de producto o miles de barriles siguiendo el ensayo de promedio representable por bootstrap \nLa región queda cercamente en segundo lugar variando en solo 2.4 unidades de producto")

Promedio de producto en región 1: 92.6437710455915, 
intervalos de confianza región 1: 95%: [92.30295971 92.95265058]

Promedio de producto en región 2: 68.85819225261552, 
intervalos de confianza región 2 95%: [68.53340479 69.1644848 ]

Promedio de producto en región 3: 95.038584189341, 
intervalos de confianza región 3 95%: [94.72110977 95.33324721]

La región 3 es la más petrolífera con 95 unidades de producto o miles de barriles siguiendo el ensayo de promedio representable por bootstrap 
La región queda cercamente en segundo lugar variando en solo 2.4 unidades de producto


Panorama del resultado de unidades promedio por región. 

Siguiendo los costos de inversión, se requieren pozos que generen más de 111 unidades de producto mientras que el promedio de la zona 3, la más petrolífera, llega a 95. 16 miles de barriles menos por pozo como promedio. 

Se debe considerar que no se seleccionará cualquier pozo sino aquellos que tengan mayor predicción de producto. 

Del análisis exploratorio se observó que hay 100,000 registros de pozos por región, se tomarán solo los 200 con mayor potencial y se revisará la viabilidad económica potencial.

In [26]:
#Cálculos de inversión y producto requerido
presupuesto = 100000000 #100 millones de dólares
pozos = 200
coste_por_pozo = presupuesto/pozos 
print(f"Inversión estimada por pozo: {coste_por_pozo}")
#objetivo 200 pozos 
#Cada barril genera 4.5 dólares y cada unidad de producto corresponde a mil barriles, es decir: 4500 dólares. 

producto_por_pozo_requerido = (presupuesto/pozos)/4500 #111 unidades de producto en miles de barriles por pozo. 
print(f"Producto por pozo requerido para satisfacer la inversion: {producto_por_pozo_requerido}")


Inversión estimada por pozo: 500000.0
Producto por pozo requerido para satisfacer la inversion: 111.11111111111111


In [21]:
#Análisis económico potencial 
prediccion_y_rmse_geo1[0] #predicciones producto de datos de validacion
prediccion_y_rmse_geo2[0]
prediccion_y_rmse_geo3[0] 

datos_divididos_geo1[3] #Datos reales de producto
datos_divididos_geo2[3]
datos_divididos_geo3[3]

#Tenemos el indicio de que la region 3 y 1 son las mejores pero igual se hará con las 3 regiones. 

#Hacer la función de selección de los 200 pozos con mayor potencial
def seleccion_top_200(predicciones, objetivos): 
    df = pd.DataFrame({
    "pred" : predicciones, 
    "real": objetivos.reset_index(drop=True)
    })
    top200 = df.sort_values(by="pred", ascending=False).head(200)
    return top200

In [25]:
#Llamar la función sobre las 3 regiones para tener los 200 mejores pozos por región con datos predichos por el modelo y reales. 
top200_region1 = seleccion_top_200(prediccion_y_rmse_geo1[0], datos_divididos_geo1[3])
top200_region2 = seleccion_top_200(prediccion_y_rmse_geo2[0], datos_divididos_geo2[3])
top200_region3 = seleccion_top_200(prediccion_y_rmse_geo3[0], datos_divididos_geo3[3])

display(top200_region1.head())
display(top200_region2.head())
display(top200_region3.head())

#Muy interesante conseguir promedios reales y predichos de nadamas los top 200
promedio_producto_real_region1 = top200_region1["real"].mean()
promedio_producto_pred_region1 = top200_region1["pred"].mean()
promedio_producto_real_region2 = top200_region2["real"].mean()
promedio_producto_pred_region2 = top200_region2["pred"].mean()
promedio_producto_real_region3 = top200_region3["real"].mean()
promedio_producto_pred_region3 = top200_region3["pred"].mean()

promedios_pred_y_real_por_region = pd.DataFrame({
    "region": ["region1", "region2", "region3"], 
    "promedio_real":  [promedio_producto_real_region1, promedio_producto_real_region2, promedio_producto_real_region3], 
    "promedio_pred": [promedio_producto_pred_region1, promedio_producto_pred_region2, promedio_producto_pred_region3]
})

promedios_pred_y_real_por_region = promedios_pred_y_real_por_region.round(2)
print(promedios_pred_y_real_por_region)

print("\nPodemos observar de los promedios de predicción y reales, que la zona 1 contiene pozos de mayor rendimiento en producto potencial. Alrededor de 7 unidades por pozo, 7000 barriles.")

,pred,real
9317,180.180713,162.810993
219,176.252213,153.639837
10015,175.850623,162.153488
11584,175.658429,96.893581
23388,173.299686,178.879516


,pred,real
20430,139.818970,137.945408
7777,139.773423,137.945408
8755,139.703330,137.945408
1178,139.560938,137.945408
4285,139.516754,137.945408


,pred,real
22636,165.856833,175.103291
24690,165.679685,131.627481
7811,163.439962,141.160070
1581,162.062589,159.676082
6751,161.797476,142.135203


    region  promedio_real  promedio_pred
0  region1         148.01         155.51
1  region2         137.95         138.73
2  region3         141.23         148.02

Podemos observar de los promedios de predicción y reales, que la zona 1 contiene pozos de mayor rendimiento en producto potencial. Alrededor de 7 unidades por pozo, 7000 barriles.


In [31]:
#Calcular las ganancias por región usando los datos de predicción y reales.
def ingresos_por_region(top200):
    ganancia_unidad = 4500 #1000 barriles de 4.5 dolares de ingreso cada uno.  
    coste_por_pozo = 500000
    ingresos = top200["real"] * ganancia_unidad
    ganancia_total = ingresos.sum() - (coste_por_pozo*len(top200))
    ganancia_total = ganancia_total.round(2)
    ingresos_potenciales = top200["pred"]*ganancia_unidad
    ingresos_pred = top200["pred"] * ganancia_unidad
    ganancia_total_potencial = ingresos_pred.sum() - (coste_por_pozo*len(top200))
    ganancia_total_potencial = ganancia_total_potencial.round(2)
    return ganancia_total, ganancia_total_potencial

In [40]:
#Ejecutar la función de ganancia sobre los 200 mejores pozos por región. 
ganancias_region1 = ingresos_por_region(top200_region1)
ganancias_region2 = ingresos_por_region(top200_region2)
ganancias_region3 = ingresos_por_region(top200_region3)

print(ganancias_region1)
print(f"Ganancia total en la región 1 sería de {(ganancias_region1[0]/1000000):.2f} millones de dólares y {(ganancias_region1[1]/1000000):.2f} millones de dólares conforme a lo predicho por el modelo.")
print(ganancias_region2)
print(f"Ganancia total en la región 2 sería de {(ganancias_region2[0]/1000000):.2f} millones de dólares y {(ganancias_region2[1]/1000000):.2f} millones de dólares según las predicciones.")
print(ganancias_region3)
print(f"Ganancia total en la región 3 sería de {(ganancias_region3[0]/1000000):.2f} millones de dólares y {(ganancias_region3[1]/1000000):.2f} millones de dólares según las predicciones.")

print("\nLa ganancia de acuerdo a los datos reales de producto en pozos por región y predichos, la región 1 sería la de mayor viabilidad económica. Generando ingresos (adicionales a la inversión) de 33 a ~40 millones de dólares. \nA pesar de que el promedio de producto es mayor en la región 3, los 200 valores mayores para producto en pozo pertenecen a la región 1.")

(np.float64(33208260.43), np.float64(39960488.77))
Ganancia total en la región 1 sería de 33.21 millones de dólares y 39.96 millones de dólares conforme a lo predicho por el modelo.
(np.float64(24150866.97), np.float64(24857120.52))
Ganancia total en la región 2 sería de 24.15 millones de dólares y 24.86 millones de dólares según las predicciones.
(np.float64(27103499.64), np.float64(33217543.96))
Ganancia total en la región 3 sería de 27.10 millones de dólares y 33.22 millones de dólares según las predicciones.

La ganancia de acuerdo a los datos reales de producto en pozos por región y predichos, la región 1 sería la de mayor viabilidad económica. Generando ingresos (adicionales a la inversión) de 33 a ~40 millones de dólares. 
A pesar de que el promedio de producto es mayor en la región 3, los 200 valores mayores para producto en pozo pertenecen a la región 1.


In [42]:
#bootstrapping. Distribución de Ganancias potenciales por región. 
def bootstrap_ganancia(dftop200, n_iter = 1000, seed= 123): 
    """_summary_
    Esta función genera muestras a partir de un marco de datos para concentrar la distribución como medida central. 
    Args:
        dftop200 (_type_): marco de datos a partir del cual se hacen los muestreos.
        n_iter (int, optional): Cantidad de muestras. Defaults to 1000.
        seed (int, optional): componente aleatorizador que garantiza reproducibilidad. Defaults to 123.

    Returns:
        _type_: _description_
    """
    np.random.seed(seed)
    ganancias = []
    ganancia_barril = 4500
    for i in range(n_iter): 
        muestra = resample(dftop200["real"], replace=True, n_samples=200)
        ingresos = muestra * ganancia_barril
        ganancia = ingresos.sum() - (coste_por_pozo*200)
        ganancias.append(ganancia)
        
    ganancias = np.array(ganancias)
    promedio = np.mean(ganancias)
    intervalo = np.percentile(ganancias, [2.5, 97.5])
    riesgo = np.mean(ganancias < 0) * 100  # porcentaje con pérdida
    return promedio, intervalo, riesgo

#Ejecutar la función
promedio_ganancias_region1 = bootstrap_ganancia(top200_region1)
promedio_ganancias_region2 = bootstrap_ganancia(top200_region2)
promedio_ganancias_region3 = bootstrap_ganancia(top200_region3)

print(f"Promedio de ganancias region 1: {(promedio_ganancias_region1[0])/1000000:.2f}, IC 95% {promedio_ganancias_region1[1]}, riesgo: {promedio_ganancias_region1[2]}")
print(f"Promedio de ganancias region 2: {(promedio_ganancias_region2[0])/1000000:.2f}, IC 95% {promedio_ganancias_region2[1]}, riesgo: {promedio_ganancias_region2[2]}")
print(f"Promedio de ganancias region 3: {(promedio_ganancias_region3[0])/1000000:.2f}, IC 95% {promedio_ganancias_region3[1]}, riesgo: {promedio_ganancias_region3[2]}")
#Selección de región de pozos. 

Promedio de ganancias region 1: 33.15, IC 95% [30115387.61849997 36111280.96263297], riesgo: 0.0
Promedio de ganancias region 2: 24.15, IC 95% [24150866.96681508 24150866.96681508], riesgo: 0.0
Promedio de ganancias region 3: 27.21, IC 95% [23578287.31959873 30797265.11531992], riesgo: 0.0


Resultados muy similares del bootstrap. 

Region 1 tiene un promedio post bootstrap o muestreo de extensión de 33 millones de dólares de ganancia. 
Rango de ganancia de acuerdo a intérvalos con un 95% de confianza desde 30.1 millones de dólares hasta 36.1 millones de dólares. 

Region 3 en el rango de los 27.21 millones de dólares de ganancia promedio. Con un rango de 95% de confianza desde los 23.5 a ~30.8 millones de dólares. 

In [47]:
#Obtención de los Id de los pozos más relevantes para extracción de barriles de crudo. 
datos_geo1_seleccionado_top200 = datos_geo1[datos_geo1["product"].isin(top200_region1["real"])]
display(datos_geo1_seleccionado_top200)
print("Datos de los pozos a generar.")

datos_geo1_seleccionado_top200_id = datos_geo1_seleccionado_top200["id"]

,id,f0,f1,f2,product
1092,3WINl,1.428935,-0.379902,9.550830,127.540144
1323,mG8fp,0.448925,-0.339043,9.986641,181.857009
1818,IW3E9,1.970622,0.175233,10.805434,107.114081
2430,QcEBB,1.108474,-0.152508,9.874963,149.912017
2570,y1pfT,0.477885,-0.391866,10.439256,168.337706
...,...,...,...,...,...
97925,ruR8J,0.951343,0.212148,11.670685,140.115058
98105,CBsG2,1.841304,-0.404711,10.244346,149.725360
98799,MDQtM,0.527002,-0.292326,9.799607,95.396917
99065,OXpa9,1.412223,-0.374404,10.208000,42.092314


Datos de los pozos a generar.


Conclusiones

Se generó un modelo predictivo de la cantidad de producto en pozos petroleros a partir de datos reales. 

Se estimó la predicción promedio de producto por región con intervalos de confianza. 

Se empleó muestreo expansivo de tipo bootstrapping para aumentar la muestra e identificar valores tendencia relevantes como producto promedio por pozo por región. 

Se identificó que aunque la región 3 contiene un promedio mayor de producto por pozo, al organizar los pozos por aquellos de mayor valor de producto real y predicho los valores mayores pertenecen a la región 1. 

Se recomienda generar los pozos petrolíferos en la región 1, los pozos de mayor producto se guardaron en la variable datos_geo1_seleccionado_top200. 